In [20]:
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from kerastuner import HyperModel, Hyperband


In [21]:
# Plan
# 1. interchanged x1 and x2 based on their Value
# removed column leak_1
# create a model first with 4 coordinate - 2 for each. 
# Then use that model as base model, add additional layers for leakageness and do transfer learning to do the 6 output case


In [22]:
model_path = 'saved_model/Multi_leak/2_loss/Mid_structure_anchor/4 Output_to_transfer/'
project_name='multi_task_tuning_4coords_totransfer'

In [23]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
# two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
# use the idea of masking instead
data['x2'] = data['x2'].replace(np.nan, 8024)
data['y2'] = data['y2'].replace(np.nan, 2616.5)

In [24]:
data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'tot_residual_flow', 
       'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[[ 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[[ 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[[ 'leak_2']]


# y1_test.to_csv('check1.csv')

y1_data = [y1_train, y1_val, y1_test]
y1_data_types = ['y1_train', 'y1_val', 'y1_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

# print()

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

# y1_test.to_csv('check2.csv')


In [ ]:

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

In [5]:
# y1_train['x2'] = y1_train['x2'].replace(np.nan, -5)
# y1_train['y2'] = y1_train['y2'].replace(np.nan, -5)

# y1_test['x2'] = y1_test['x2'].replace(np.nan, -5)
# y1_test['y2'] = y1_test['y2'].replace(np.nan, -5)

# y1_val['x2'] = y1_val['x2'].replace(np.nan, -5)
# y1_val['y2'] = y1_val['y2'].replace(np.nan, -5)
# Not sure if 0 is good enough or try generating a random number

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

# y2_train = pd.DataFrame(y2_train, columns=y2_columns)
# y2_test = pd.DataFrame(y2_test, columns=y2_columns)
# y2_val = pd.DataFrame(y2_val, columns=y2_columns)

y2_train = y2_train.reset_index().drop(columns='sample_number')
y2_val = y2_val.reset_index().drop(columns='sample_number')
y2_test = y2_test.reset_index().drop(columns='sample_number')

# y_train = pd.concat([y1_train, y2_train], axis=1)
# y_test = pd.concat([y1_test, y2_test], axis=1)
# y_val = pd.concat([y1_val, y2_val], axis=1)

# scaler_flows = StandardScaler()
# X_train = scaler_flows.fit_transform(X_train)
# X_test = scaler_flows.transform(X_test)
# X_val = scaler_flows.transform(X_val)

In [6]:
X_train_np, y1_train_np = X_train.values, y1_train.values,
X_val_np, y1_val_np = X_val.values, y1_val.values
X_test_np, y1_test_np = X_test.values, y1_test.values
# Create TensorFlow datasets from NumPy arrays.
batch_size = 32
buffer_size = len(X_train)  # Set the buffer size to the number of training examples for full shuffling.

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y1_train_np ))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y1_val_np))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y1_test_np))

# Shuffle, batch, and prefetch the training dataset.
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation and test datasets.
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [7]:
# y_train = {
#     "y1" : y1_train,
#     "y2" : y2_train
# }

# y_val = {
#     "y1" : y1_val,
#     "y2" : y2_val
# }

# y_test = {
#     "y1" : y1_test,
#     "y2" : y2_test
# }

# losses = {
# 	"y1": "mse",
# 	"y2": 'binary_crossentropy'
#     # "y2" : 'mse'
#     }

# metrics = {
#     "y1": 'mae',
#     "y2": 'mae'
#     }

losses = "mse"
metrics = 'mae'

In [8]:
# Experiment 2 with Nadam

EPOCHS = 1000
# Define the multi-task HyperModel
class MultiTaskHyperModel(HyperModel):

    def build(self, hp):
        inputs = keras.Input(shape=(10,))
        shared_layer = inputs
        for i in range(hp.Int('num_layers', 1, 15)):
            shared_layer = layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", 'elu']),
                kernel_initializer='he_uniform'
            )(shared_layer)


        task_layer1 = shared_layer
        # for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
        #     task_layer1 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]), 
        #                                activation='relu',
        #                                kernel_initializer='he_uniform')(task_layer1)
        y1 = layers.Dense(4, name='y1', activation = 'linear', kernel_initializer='he_uniform')(task_layer1)

        # task_layer2 = shared_layer
        # for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
        #     task_layer2 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]),
        #                                 activation='relu',
        #                                 kernel_initializer='he_uniform')(task_layer2)
        # y2 = layers.Dense(2, name='y2', activation = 'sigmoid', kernel_initializer='he_uniform')(task_layer2)

        outputs = y1

        # loss1_weight = hp.Float("loss1_weight", min_value=1e-4, max_value=1, sampling="log")
        # loss1_weight = hp.Float("loss2_weight", min_value=1e-4, max_value=1, sampling="log")
        
        # loss1_weight = hp.Choice('loss1_weight', values=[0.2, 0.4, 0.6, 0.8])
        # # loss2_weight = hp.Choice('loss2_weight', values=[0.00005, 0.2, 0.4, 0.6, 0.8, 1.0])
        # # to stop the model from reducing both these weightage to very low and hence reducing the total loss
        # loss2_weight = 1 - loss1_weight

        # lossWeights = {"y1": loss1_weight, "y2": loss2_weight}

        model = keras.Model(inputs=inputs, outputs=outputs)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

        # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        #             loss="mse",  metrics='mae')
        
        model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                    loss=losses, 
                    # loss_weights=lossWeights,
                    metrics = metrics)
        return model

# Create the multi-task HyperModel
multi_task_hypermodel = MultiTaskHyperModel()

# Define the Hyperband tuner
tuner = Hyperband(
    multi_task_hypermodel,
    objective = kt.Objective("val_loss", direction="min"),
    max_epochs=EPOCHS,
    factor=2,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name=project_name
)

# Perform hyperparameter search
tuner.search(train_dataset.map(lambda x, y1: (x, {'y1': y1})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1: (x, {'y1': y1})), 
             verbose = 1,
             epochs=EPOCHS, shuffle = True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model on the full dataset
best_model.fit(train_dataset.map(lambda x, y1: (x, {'y1': y1})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1: (x, {'y1': y1})), 
             verbose = 1, epochs=EPOCHS, shuffle = True)

INFO:tensorflow:Reloading Tuner from ../../tensorflow_log_files/studienarbeit/multi_task_tuning_4coords_totransfer/tuner0.json
INFO:tensorflow:Oracle triggered exit
Best Hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7fe426a5f8e0>
Epoch 1/1000
20/20 [==============================] - 4s 23ms/step - loss: 2.4367 - mae: 1.0568 - val_loss: 0.4281 - val_mae: 0.4947
Epoch 2/1000
20/20 [==============================] - 0s 14ms/step - loss: 0.2653 - mae: 0.3775 - val_loss: 0.2140 - val_mae: 0.3345
Epoch 3/1000
20/20 [==============================] - 0s 15ms/step - loss: 0.1783 - mae: 0.3043 - val_loss: 0.2297 - val_mae: 0.3539
Epoch 4/1000
20/20 [==============================] - 0s 14ms/step - loss: 0.1633 - mae: 0.2870 - val_loss: 0.2257 - val_mae: 0.3322
Epoch 5/1000
20/20 [==============================] - 0s 14ms/step - loss: 0.1558 - mae: 0.2801 - val_loss: 0.1377 - val_mae: 0.2718
Epoch 6/1000
20/20 [==============================] - 

In [9]:
best_model.save(model_path)

INFO:tensorflow:Assets written to: saved_model/Multi_leak/2_loss/Mid_structure_anchor/4 Output_to_transfer/assets


INFO:tensorflow:Assets written to: saved_model/Multi_leak/2_loss/Mid_structure_anchor/4 Output_to_transfer/assets


In [10]:
best_model.summary()
y_predictions = best_model.predict(test_dataset.map(lambda x, y1: (x, {'y1': y1})))
results_train = best_model.evaluate(train_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_val = best_model.evaluate(val_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_test = best_model.evaluate(test_dataset.map(lambda x, y1: (x, {'y1': y1,})), verbose=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 352)               3872      
                                                                 
 dense_1 (Dense)             (None, 32)                11296     
                                                                 
 dense_2 (Dense)             (None, 384)               12672     
                                                                 
 dense_3 (Dense)             (None, 480)               184800    
                                                                 
 dense_4 (Dense)             (None, 160)               76960     
                                                                 
 dense_5 (Dense)             (None, 480)               77280 

In [11]:
# y_pred = np.concatenate((y_predictions[0],y_predictions[1]), axis=1)
# y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
# y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
# y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
# y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1
y_pred = pd.DataFrame(y_predictions, columns=y1_columns)
# y1_pred_inverse = scaler_coords1.inverse_transform(y_pred[['x1', 'y1','x2', 'y2']])
y1_pred_inverse = scaler_coords1.inverse_transform(y_predictions)
y_pred[['x1', 'y1','x2', 'y2']] = pd.DataFrame(y1_pred_inverse,columns=['x1', 'y1','x2', 'y2'])
pd.concat([y_pred, y_test.reset_index().drop(columns='sample_number')], axis=1).to_csv(model_path+'predictions.csv')

In [12]:
y1_pred_inverse

array([[ 6742.95 ,  2179.576, 14809.613,  4000.318],
       [ 5272.015,  1109.384, 10282.072,  1526.595],
       [ 2388.986,  3003.733,  8659.594,  3114.507],
       [ 4403.101,  2349.979,  9074.645,  3805.012],
       [ 3689.475,  2112.742, 12867.874,  2588.684],
       [ 7123.04 ,  4169.128,  8036.048,  2632.877],
       [10096.364,  1593.33 , 11695.624,  1139.946],
       [ 7121.706,  1612.259,  8050.839,  2617.347],
       [ 7587.076,  1393.601, 10687.264,  2044.726],
       [ 8188.496,  2678.728, 10128.402,  4517.895],
       [ 8192.633,  2724.522, 14748.47 ,  2477.789],
       [ 8260.257,  2702.897, 11423.363,  1081.718],
       [ 8303.69 ,  2033.652, 12772.013,  4073.719],
       [ 5582.571,  4429.226, 10990.655,  1443.315],
       [ 4781.996,  4445.876,  6328.174,  4295.454],
       [ 3204.87 ,  4209.585,  5724.113,  3182.798],
       [ 8955.   ,  3294.798, 14904.357,  2420.608],
       [ 2927.186,  1832.215,  9777.786,  1786.491],
       [ 5040.151,  3448.668,  8063.532,  2608

In [13]:
from sklearn.metrics import mean_squared_error

mse_x1 = mean_squared_error(y_test['x1'], y_pred['x1'], squared=True)
mse_x2 = mean_squared_error(y_test['x2'], y_pred['x2'], squared=True)

mse_y1 = mean_squared_error(y_test['y1'], y_pred['y1'], squared=True)
mse_y2 = mean_squared_error(y_test['y2'], y_pred['y2'], squared=True)

# mse_leak1 = mean_squared_error(y_test['leak_1'], y_pred['leak_1'], squared=True)
# mse_leak2 = mean_squared_error(y_test['leak_2'], y_pred['leak_1'], squared=True)

mse = [mse_x1, mse_x2, mse_y1, mse_y2]

In [14]:
mse

[14602710.822075196, 14447919.378391817, 1316183.277413952, 1127163.25369344]

In [15]:


# idea to explore - 
# 1. Just use 1 loss function for the entire output
# 2. Give 2 loss function for each leakageness and coordinates (2a. Try the same without sigmoid function)
# 3.
# create a model first with 4 coordinate - 2 for each. 
# Then use that model as base model, add additional layers for leakageness and do transfer learning to do the 6 output case

# 4. 
# use masking feature of tensorflow (low priority) - # Use the idea of masking for the case where when we have to only 1 leakage 

# do i actually need objectness 1. since it is always present.i just need 1 objectness right ?
# coordinate are more important right - hence give more wightage to the coordinate loss function
# try auxiliary task and joint model
# remove bad quality data

In [ ]:
# swapping x1 and x2 in ascending order does improves accuracy
# found mistake in experiment 3 - that nan values in single leakage case were not actually updated.
# it was still blank. i suspect that it was assumed to be zero during the scaling process and scaled with the zero value.